<a href="https://colab.research.google.com/github/Harshpatel44/Softmax-classifier-Network/blob/master/softmax-classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
def single_from_one_hot(z):
    return np.argmax(z,axis=1)

In [0]:
def plot_decision(obj,X,numOutputs):
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.1), np.arange(y_min, y_max, 0.1))
    ar=np.array([xx.ravel(), yy.ravel()])   
    for i in range(numOutputs+1):   # adding column for bias and label for consistency
      ar=np.append(ar,np.zeros((1,ar.shape[1]))+1,axis=0)
    z = obj.predict(ar.T,1)
    z = z[:,0].reshape(xx.shape)
    plt.contourf(xx,yy,z)
    plt.scatter(X[:,0],X[:,1],c=X[:,-1])
    plt.show()

In [0]:
def train_test_split(data,tr_percent,te_percent):
    n=np.round((data.shape[0] * tr_percent) / 100)
    return data[0:int(n),:-1],data[0:int(n),-1],data[int(n):,:-1],data[int(n):,-1]

# Ek Naya Daur

In [0]:
from sklearn import datasets
import numpy as np
mnist = datasets.fetch_openml('mnist_784')

In [0]:
def one_hot_encoding(X,classes):
  one_hot_X=np.zeros((X.shape[0],classes))
  one_hot_X[range(X.shape[0]),np.array(X[:,-1],dtype='int8')] = 1
  X=np.array(X[:,:-1])
  X=np.append(X,one_hot_X,axis=1)
  return X

In [0]:
train = mnist.data[:60000,:]
test =  mnist.data[60000:,:]
train_y = mnist.target[:60000]
test_y =  mnist.target[60000:]


train_y=train_y.reshape((train_y.shape[0],1))
test_y=test_y.reshape((test_y.shape[0],1))
train_y= np.array(train_y,dtype="float32")
test_y= np.array(test_y,dtype="float32")


train_y=one_hot_encoding(train_y,10)
test_y=one_hot_encoding(test_y,10)
train=np.clip(train_data,0,1)
test=np.clip(test_data,0,1)

In [32]:
print(train_y[0])

[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


In [71]:
class Model():
    def __init__(self):
        self.hidden_units=5
        self.train=train
        self.learning_rate=0.01
        self.output_classes=10
        
    def init_weights(self):
        self.w1=np.random.rand(self.train.shape[1],self.hidden_units)
        self.b1=np.zeros((self.train.shape[0],self.hidden_units))
        self.w2=np.random.rand(self.hidden_units,self.hidden_units)
        self.b2=np.zeros((self.train.shape[0],self.hidden_units))
        self.w3=np.random.rand(self.hidden_units,self.output_classes)
        self.b3=np.zeros((self.train.shape[0],self.output_classes))

    def loss(self,y,t):
        return -np.sum(t * np.log(y))

    def activ(self,z):
        return np.exp(z - np.max(z,axis=1,keepdims=True))/np.sum(np.exp(z-np.max(z,axis=1,keepdims=True)))

    def deriv_activ(self,z,n='r'):
        if(n=="r"):
            return np.where(z>0,1,0)

    def train_model(self,train,train_y):
        
        for i in range(5000):
          
            h1=self.activ(np.dot(self.train,self.w1,)+self.b1)
            h2=self.activ(np.dot(h1,self.w2)+self.b2)
            y=self.activ(np.dot(h2,self.w3)+self.b3)
            l=self.loss(y,train_y)

            print(l)
            # derivative for softmax and cross entropy loss together
            dl = y - train_y                             # [60k x 10]

            #w3/dl
            w3_ = np.dot(h2.T,dl)                        # [5 x 60k] x [60k x 10] = [5 x 10] 
            b3_ = dl

            #w2/dl
            dz3_=np.dot(dl,self.w3.T)                    # [60k x 10] x [10 x 5] = [60k x 5]
            dh2_=np.multiply(dz3_,self.deriv_activ(h2))  # [60k x 5] * [60 x 5]  = [60k x 5]
            w2_=np.dot(h1.T,dh2_)                        # [5 X 60k] x [60k x 5] = [5 x 5]
            b2_=dh2_

            #w1/dl
            dz2_=np.dot(dh2_,self.w2.T)                   # [60k x 5] x [5 x 5] = [60k x 5]
            dh1_ =np.multiply(dz2_,self.deriv_activ(h1))  # [60k x 5] * [60k x 5] = [60k x 5]
            w1_=np.dot(self.train.T,dh1_)                 # [784 x 60k] * [60k x 5] = [784 x 5]
            b1_=dh2_
         
            # self.w3+= np.divide( (-self.learning_rate * w3_) , np.max(self.w3,axis=1,keepdims=True) )
            # self.w2+= np.divide( (-self.learning_rate * w2_) , np.max(self.w2,axis=1,keepdims=True) )
            # self.w1+= np.divide( (-self.learning_rate * w1_) , np.max(self.w1,axis=1,keepdims=True) )
            
            self.w3+= (-self.learning_rate * w3_)
            self.w2+= (-self.learning_rate * w2_)
            self.w1+= (-self.learning_rate * w1_)
            
            self.b3+= (-self.learning_rate * b3_) 
            self.b2+= (-self.learning_rate * b2_) 
            self.b1+= (-self.learning_rate * b1_) 
    def test_model(self,test,test_y)


net=Model()
net.train_model(train,train_y)
net.test_model(test,test_y)

798281.0965501452
797741.3665407997
797202.1808772808
796663.5439170689
796125.4600422075
795587.9336594472
795050.9692002129
794514.5711205417
793978.7439010338
793443.4920467808
792908.8200873112
792374.732576511
791841.2340925521
791308.32923781
790776.0226387917
790244.3189460273
789713.2228339945
789182.7390010075
788652.8721691234
788123.6270840231
787595.0085149137
787067.0212543901
786539.6701183311
786012.9599457544
785486.8955986936
784961.4819620535
784436.7239434664
783912.6264731406
783389.1945037106
782866.4330100623
782344.3469891819
781822.9414599736
781302.2214630772
780782.1920606972
780262.8583363987
779744.2253949197
779226.2983619624
778709.0823839927
778192.5826280158
777676.8042813615
777161.7525514591
776647.4326655993
776133.8498706957
775621.0094330405
775108.9166380469
774597.5767900018


KeyboardInterrupt: ignored